In [45]:
import pickle as pkl
with open('./resources/covid_en_tweet.pickle', 'rb') as pkl_in:
    tweets_en = pkl.load(pkl_in)
#loading bengali data
with open('./resources/covid_bn_tweet.pickle', 'rb') as pkl_in:
    tweets_bn = pkl.load(pkl_in)
with open('./resources/covid_bn_tweet_test.pickle', 'rb') as pkl_in:
    tweets_bn_test = pkl.load(pkl_in)
#loading hindi data
with open('./resources/covid_hi_tweet.pickle', 'rb') as pkl_in:
    tweets_hi = pkl.load(pkl_in)

In [57]:
#textural features
def get_text_ft(tweets):
    text_ft = []
    len_ = tweets['text_info']['len']
    upp = tweets['text_info']['num_of_upper_char']
    qmark = tweets['text_info']['num_of_qmarks']
    exclm = tweets['text_info']['num_of_exclmmarks']
    urls = tweets['text'].apply(lambda x: x.count('URL'))
    for i in range(len(tweets['text'])):
        #print(i)
        text_ft.append([len_[i], upp[i], qmark[i], exclm[i], urls[i]])    
    return text_ft

In [59]:
text_ft_en = get_text_ft(tweets_en)
#text_ft_en

In [2]:
import pandas as pd
del tweets_en['text_info']
del tweets_bn['text_info']
del tweets_bn_test['text_info']
del tweets_hi['text_info']
df_en = pd.DataFrame(tweets_en)
df_bn = pd.DataFrame(tweets_bn)
df_bn_test = pd.DataFrame(tweets_bn_test)
df_hi = pd.DataFrame(tweets_hi)
print(df_en)

                                                  text  labels
0    for the average american the best way to tell ...       0
1                             this is fucking bullshit       0
2    can y ’ all please just follow the government ...       0
3    no offense but the corona virus disappearing b...       0
4    this is the face of someone who just spent  9 ...       1
..                                                 ...     ...
499  president trump ' s comments about the coronav...       1
500  drug companies reportedly killed a provision i...       1
501  does @ twitter have a team fighting co vid  - ...       0
502  fox to cause pandemic  fox news is spreading  ...       1
503  nurtw lagos is already sens it i zing their pe...       1

[504 rows x 2 columns]


In [24]:
#df_merged = pd.concat([df_en, df_bn, df_hi, df_bn_test])
df_merged = df_en
from sklearn.model_selection import train_test_split
train_df, eval_df = train_test_split(df_merged, test_size=0.2)
eval_df, test_df = train_test_split(eval_df, test_size=0.5)

train_data = train_df['text'].to_list()
eval_data = eval_df['text'].to_list()
test_data = test_df['text'].to_list()

train_tags = train_df['labels'].to_list()
eval_tags = eval_df['labels'].to_list()
test_tags = test_df['labels'].to_list()

In [25]:
assert len(train_data) == len(train_tags)
assert len(eval_data) == len(eval_tags)
assert len(test_data) == len(test_tags)

print('Length of train, valid, test = ', len(train_data), len(eval_data), len(test_data))
print('Samples : ', test_data[0], test_tags[0])

Length of train, valid, test =  403 50 51
Samples :  a state of emergency has been declared for la city and county due to confirmed cases of coronavirus . angelenos need to be prepared , but not panicked . listen to the advice of experts . make sure you ' re following @cdcgov and @la public health for the latest info and recommendations . <link> 1


In [26]:
import pickle as pkl
with open('./resources/tweets_preprocessed_multi.pickle', 'wb') as pkl_out:
    pkl.dump(train_data, pkl_out)
    pkl.dump(eval_data, pkl_out)
    pkl.dump(test_data, pkl_out)
    
    pkl.dump(train_tags, pkl_out)
    pkl.dump(eval_tags, pkl_out)
    pkl.dump(test_tags, pkl_out)

In [6]:
from flair.data import Sentence
from flair.embeddings import BertEmbeddings
embeddings = BertEmbeddings('bert-base-multilingual-uncased', layers='-1') # use only last layers
import numpy as np
def get_word_embeddings(sent_list):
    
    word_embeds = {}
    for i, sent in enumerate(sent_list):
        sentence = Sentence(sent)
        embeddings.embed(sentence)
        for word in sent.split():
            try:
                word_embeds[word+ '_' + str(i)] = [np.asarray(token.embedding) for token in sentence if str(token).split(' ')[-1].strip() == word.strip()][0]
            except Exception as e:
                print(str(e), word)
                word_embeds[word+ '_' + str(i)] = np.zeros(768)
    
    return word_embeds

/home/debanjana_ibm/anaconda3/envs/covidfake/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
words = get_word_embeddings(train_data + eval_data + test_data)

list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index out of range 
list index o

In [28]:
words_list = list(words.values())

In [29]:
with open('./resources/tweet_word_emb_en.pickle', 'wb') as pkl_out:
    pkl.dump(words_list, pkl_out)

In [10]:
with open('./resources/keyphrases_embeddings.pickle', 'rb') as pkl_in:
    en_embeds =  pkl.load(pkl_in)
    bn_embeds1 = pkl.load(pkl_in)
    bn_embeds2 = pkl.load(pkl_in)
    hi_embeds = pkl.load(pkl_in)

In [11]:
import numpy as np
keyphr_embeds = en_embeds + bn_embeds1 + hi_embeds +  bn_embeds2
keyphr_embeds = np.asarray([emb.detach().numpy() for emb in keyphr_embeds])
keyphr_embeds.shape

(1333, 768)

In [30]:
with open('./resources/multi_raw_outputs.pickle', 'rb') as pkl_in:
    wt_dict = pkl.load(pkl_in)
#print(wt_dict)

In [13]:
from simpletransformers.classification import ClassificationModel
import sklearn
import torch

model = torch.load('./models/multi_model')
result, model_outputs, wrong_predictions = model.eval_model(df_bn_test, f1=sklearn.metrics.f1_score, acc=sklearn.metrics.accuracy_score)
model_outputs

array([[-2.572136  ,  2.574471  ],
       [-2.4838808 ,  2.4175797 ],
       [-2.357317  ,  2.3285203 ],
       [-2.3505762 ,  2.2907462 ],
       [-0.50719357,  0.46243185],
       [-2.5837748 ,  2.526768  ],
       [-2.5599816 ,  2.4843316 ],
       [ 1.4463357 , -1.3085309 ],
       [ 1.1202005 , -1.0068094 ],
       [ 1.3785052 , -1.240451  ],
       [-2.5774004 ,  2.5625243 ],
       [-0.30324373,  0.27542344],
       [-2.5102234 ,  2.4450874 ],
       [-2.5673108 ,  2.5722244 ],
       [-1.9969274 ,  1.9005226 ],
       [-2.5359445 ,  2.5432196 ],
       [-2.1424928 ,  2.0144618 ],
       [ 1.3975838 , -1.3008662 ],
       [ 1.3536893 , -1.238526  ],
       [-2.4330568 ,  2.3929024 ],
       [-2.1610408 ,  2.1246333 ],
       [-2.487532  ,  2.4530842 ],
       [-1.3435651 ,  1.1356248 ],
       [-1.5721275 ,  1.5309868 ],
       [ 1.7831495 , -1.5740577 ],
       [ 0.9207386 , -0.8900965 ],
       [-1.7828465 ,  1.5735496 ],
       [-2.5099256 ,  2.5033908 ],
       [-2.1470275 ,

In [33]:
print(len(en_embeds))

504


In [37]:
from scipy.special import softmax
feat_embeds = []
for key, value in wt_dict.items():
    for idx, num in enumerate(value['indices']):
        if num < len(en_embeds):#trick to get indices you want
            feat_embeds.append(softmax(value['outputs'][idx]))
feat_embeds = np.asarray(feat_embeds)
feat_embeds.shape

(504, 2)

In [15]:
feat_embeds

array([[0.92410326, 0.07589663],
       [0.9467504 , 0.05324958],
       [0.9671815 , 0.03281866],
       ...,
       [0.25433308, 0.7456669 ],
       [0.02572706, 0.9742729 ],
       [0.98565495, 0.01434495]], dtype=float32)

In [16]:
#only for multilingual or bengali
model_outputs = np.asarray([softmax(wt) for wt in model_outputs])
feat_embeds = np.append(feat_embeds, model_outputs, 0)
print(feat_embeds.shape)

(1333, 2)


In [38]:
#link scores
en_scr = pd.read_csv('./resources/en_score.csv')
en_scr = en_scr['score'].to_list()
bn_scr = pd.read_csv('./resources/bn_score.csv')
bn_scr = bn_scr['score'].to_list()
hi_scr = pd.read_csv('./resources/hn_score1.csv')
hi_scr = hi_scr['score'].to_list()
bn_scr_2 = bn_scr[-100:] 
bn_scr = bn_scr[:len(bn_scr)-100]
#link_scr = en_scr + bn_scr + hi_scr + bn_scr_2
link_scr = en_scr
len(link_scr)

504

In [39]:
with open('./resources/profanity_scores.pickle', 'rb') as pkl_in:
    pscores = pkl.load(pkl_in)
pscores = pscores[:len(en_embeds)]

In [68]:
#, text_ft_en[i][0], text_ft_en[i][1], text_ft_en[i][2], text_ft_en[i][3], text_ft_en[i][4]])
#concat features
ft_emb = []
#score,
#from sklearn import preprocessing
for i, score in enumerate(pscores):
    ft_emb.append([link_scr[i], feat_embeds[i][0], feat_embeds[i][1]])
ft_emb = np.asarray(ft_emb)
print(ft_emb.shape)

(504, 3)


In [69]:
ft_emb[0]

array([0.02273516, 0.92410326, 0.07589663])

In [70]:
with open('./resources/covid_embeddings_en.pickle', 'wb') as pkl_out:
    pkl.dump(ft_emb, pkl_out)

In [21]:
with open('./resources/features.pickle', 'wb') as pkl_out:
    pkl.dump(feat_embeds, pkl_out)
    pkl.dump(pscores, pkl_out)
    pkl.dump(link_scr, pkl_out)

In [42]:
word_idx = {word : i + 1 for i, word in enumerate(words)}
with open('./resources/multi_word_idx_en.pickle', 'wb') as pkl_out:
    pkl.dump(word_idx, pkl_out)

In [43]:
sent_idx = {sent : i + 1 for i, sent in enumerate(train_data + eval_data + test_data)}
with open('./resources/multi_sent_idx_en.pickle', 'wb') as pkl_out:
    pkl.dump(sent_idx, pkl_out)